In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

# ─────────────────────────────────────────────
# 1. LOAD
# ─────────────────────────────────────────────
df = pd.read_csv("../src/data/parkinsons_updrs.csv")
print(df.shape)
print(df.dtypes)
print(df.head())

In [ ]:
# ─────────────────────────────────────────────
# 2. EDA
# ─────────────────────────────────────────────
print("=== Missing Values ===")
print(df.isnull().sum())

print("\n=== Basic Stats ===")
print(df.describe().T)

# Target distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for ax, col in zip(axes, ["motor_UPDRS", "total_UPDRS"]):
    ax.hist(df[col], bins=40, color="#2196F3", edgecolor="white", alpha=0.85)
    ax.set_title(f"Distribution of {col}", fontsize=13)
    ax.set_xlabel(col); ax.set_ylabel("Count")
plt.tight_layout(); plt.savefig("eda_target_dist.png", dpi=150); plt.show()

# Correlation heatmap
plt.figure(figsize=(16, 12))
corr = df.drop(columns=["subject#", "test_time"]).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, cmap="coolwarm", center=0,
            annot=False, linewidths=0.3, square=True)
plt.title("Feature Correlation Matrix", fontsize=14)
plt.tight_layout(); plt.savefig("eda_corr_heatmap.png", dpi=150); plt.show()

# Correlation with targets
target_corr = corr[["motor_UPDRS", "total_UPDRS"]].drop(
    ["motor_UPDRS", "total_UPDRS"]).sort_values("total_UPDRS")

fig, axes = plt.subplots(1, 2, figsize=(10, 7))
for ax, col in zip(axes, ["motor_UPDRS", "total_UPDRS"]):
    target_corr[col].plot(kind="barh", ax=ax, color=[
        "#EF5350" if v < 0 else "#42A5F5" for v in target_corr[col]])
    ax.set_title(f"Feature Correlation → {col}")
    ax.axvline(0, color="black", linewidth=0.8)
plt.tight_layout(); plt.savefig("eda_target_corr.png", dpi=150); plt.show()

# Pairplot: top 5 features most correlated with total_UPDRS
top5 = target_corr["total_UPDRS"].abs().nlargest(5).index.tolist()
sns.pairplot(df[top5 + ["total_UPDRS"]], diag_kind="kde",
             plot_kws={"alpha": 0.3, "s": 10})
plt.suptitle("Top-5 Features vs total_UPDRS", y=1.02)
plt.savefig("eda_pairplot.png", dpi=120); plt.show()

# Boxplots per subject to catch inter-subject variability
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for ax, col in zip(axes, ["motor_UPDRS", "total_UPDRS"]):
    df.boxplot(column=col, by="subject#", ax=ax, 
               flierprops=dict(markersize=2, alpha=0.3))
    ax.set_title(f"{col} per Subject")
    ax.set_xlabel("Subject #"); plt.sca(ax); plt.xticks(fontsize=6)
plt.suptitle(""); plt.tight_layout()
plt.savefig("eda_subject_boxplot.png", dpi=150); plt.show()

In [ ]:
# ─────────────────────────────────────────────
# 3. FEATURE ABLATION FOR XGBOOST
# ─────────────────────────────────────────────
TARGET    = "total_UPDRS"          # swap to motor_UPDRS if needed
DROP_COLS = ["subject#", "test_time", "motor_UPDRS", "total_UPDRS"]

X = df.drop(columns=DROP_COLS)
y = df[TARGET]
FEATURES = X.columns.tolist()

scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

X_train_sc = scaler.fit_transform(X_train)
X_test_sc  = scaler.transform(X_test)

# ── 3a. Baseline: all features ──────────────────
def fit_eval(X_tr, X_te, y_tr, y_te, feature_names=None):
    m = XGBRegressor(n_estimators=400, learning_rate=0.05,
                     max_depth=6, subsample=0.8,
                     colsample_bytree=0.8, random_state=42, n_jobs=-1)
    m.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False)
    p = m.predict(X_te)
    return {
        "model": m,
        "MAE":   mean_absolute_error(y_te, p),
        "R2":    r2_score(y_te, p),
        "features": feature_names or list(range(X_tr.shape[1]))
    }

baseline = fit_eval(X_train_sc, X_test_sc, y_train, y_test, FEATURES)
print(f"Baseline  |  MAE={baseline['MAE']:.4f}  R²={baseline['R2']:.4f}")

# ── 3b. XGBoost native feature importance ──────
importances = pd.Series(
    baseline["model"].feature_importances_, index=FEATURES
).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
importances.plot(kind="bar", color="#42A5F5", edgecolor="white")
plt.title("XGBoost Feature Importance (gain) — Baseline Model")
plt.ylabel("Importance"); plt.xticks(rotation=45, ha="right")
plt.tight_layout(); plt.savefig("ablation_importance.png", dpi=150); plt.show()

# ── 3c. Leave-One-Out Ablation ─────────────────
# Remove each feature one at a time; record MAE & R² change vs baseline
ablation_records = []

for feat in FEATURES:
    remaining = [f for f in FEATURES if f != feat]
    idx = [FEATURES.index(f) for f in remaining]
    
    res = fit_eval(
        X_train_sc[:, idx], X_test_sc[:, idx],
        y_train, y_test, remaining
    )
    ablation_records.append({
        "dropped":     feat,
        "MAE":         res["MAE"],
        "R2":          res["R2"],
        "ΔMAE":        res["MAE"] - baseline["MAE"],   # + = worse without it
        "ΔR2":         res["R2"]  - baseline["R2"],    # - = worse without it
    })

ablation_df = pd.DataFrame(ablation_records).sort_values("ΔMAE", ascending=False)
print("\n=== Leave-One-Out Ablation (sorted by ΔMAE) ===")
print(ablation_df.to_string(index=False))

# Plot ΔMAE
plt.figure(figsize=(12, 6))
colors = ["#EF5350" if v > 0 else "#66BB6A" for v in ablation_df["ΔMAE"]]
plt.bar(ablation_df["dropped"], ablation_df["ΔMAE"], color=colors, edgecolor="white")
plt.axhline(0, color="black", linewidth=0.8, linestyle="--")
plt.title("Leave-One-Out Ablation — ΔMAE when feature is removed\n"
          "(Red = model hurts without it | Green = model improves without it)")
plt.ylabel("ΔMAE vs baseline")
plt.xticks(rotation=45, ha="right"); plt.tight_layout()
plt.savefig("ablation_LOO_mae.png", dpi=150); plt.show()

# Plot ΔR²
plt.figure(figsize=(12, 6))
colors2 = ["#EF5350" if v < 0 else "#66BB6A" for v in ablation_df["ΔR2"]]
plt.bar(ablation_df["dropped"], ablation_df["ΔR2"], color=colors2, edgecolor="white")
plt.axhline(0, color="black", linewidth=0.8, linestyle="--")
plt.title("Leave-One-Out Ablation — ΔR² when feature is removed")
plt.ylabel("ΔR² vs baseline")
plt.xticks(rotation=45, ha="right"); plt.tight_layout()
plt.savefig("ablation_LOO_r2.png", dpi=150); plt.show()

# ── 3d. Cumulative Ablation (greedy forward drop) ──
# Drop least important feature one at a time, track performance
sorted_feats = importances.index.tolist()[::-1]  # ascending importance
remaining_feats = FEATURES.copy()
cumulative = [{"n_features": len(FEATURES), "MAE": baseline["MAE"],
               "R2": baseline["R2"], "dropped": None}]

for feat in sorted_feats:
    if len(remaining_feats) <= 2:
        break
    remaining_feats.remove(feat)
    idx = [FEATURES.index(f) for f in remaining_feats]
    res = fit_eval(X_train_sc[:, idx], X_test_sc[:, idx], y_train, y_test)
    cumulative.append({
        "n_features": len(remaining_feats),
        "MAE": res["MAE"], "R2": res["R2"], "dropped": feat
    })

cum_df = pd.DataFrame(cumulative)
print("\n=== Cumulative Feature Removal ===")
print(cum_df.to_string(index=False))

fig, ax1 = plt.subplots(figsize=(12, 5))
ax2 = ax1.twinx()
ax1.plot(cum_df["n_features"], cum_df["MAE"], "o-", color="#EF5350", label="MAE")
ax2.plot(cum_df["n_features"], cum_df["R2"],  "s--", color="#42A5F5", label="R²")
ax1.invert_xaxis()
ax1.set_xlabel("Number of Features Remaining")
ax1.set_ylabel("MAE", color="#EF5350")
ax2.set_ylabel("R²",  color="#42A5F5")
ax1.axvline(cum_df.loc[cum_df["MAE"].idxmin(), "n_features"],
            color="gray", linestyle=":", label="Best MAE point")
plt.title("Cumulative Ablation — Performance vs Feature Count\n"
          "(Features removed in order of lowest → highest importance)")
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.88))
plt.tight_layout(); plt.savefig("ablation_cumulative.png", dpi=150); plt.show()

# ── 3e. Summary: recommended feature subset ────
best_n = int(cum_df.loc[cum_df["MAE"].idxmin(), "n_features"])
best_row = cum_df[cum_df["n_features"] == best_n].iloc[0]
dropped_up_to_best = sorted_feats[:len(FEATURES) - best_n]
recommended = [f for f in FEATURES if f not in dropped_up_to_best]

print(f"\n✅ Recommended feature subset ({best_n} features)")
print(f"   MAE={best_row['MAE']:.4f}  R²={best_row['R2']:.4f}")
print("   Features:", recommended)